In [2]:
import pandas as pd
from contextlib import closing
import json
import pandas as pd
import re
import requests
import snowflake.connector
import time
from snowflake.connector.pandas_tools import write_pandas

In [3]:
df = pd.read_excel("ias_q1.xlsx", sheet_name = 0)

In [4]:
df.head()

,VIDEO_ID,CHANNEL_ID,VIDEO_CATEGORY,RISK_TIER,ZEFR_MAPPING
0,xr9GGusfrW4,UCpB959t8iPrxQWj7G6n0ctQ,Entertainment,moderate,NaN
1,ptvqJBL4ljA,UCl6UMZgAD7EDCNTB8Y-ICiQ,Education,moderate,NaN
2,_-3v1MonI54,UCNIFiHaLZkYASaWDdkC1njg,Entertainment,moderate,NaN
3,kPg3M4C9N9w,UCmBA_wu8xGg1OfOkfW13Q0Q,Music,moderate,NaN
4,ws00k_lIQ9U,UCmBA_wu8xGg1OfOkfW13Q0Q,Music,moderate,NaN


In [5]:
df1 = df[['VIDEO_ID', 'CHANNEL_ID', 'VIDEO_CATEGORY', 'RISK_TIER']]

In [7]:
df1['RISK_TIER'].unique()

array(['moderate', 'high risk'], dtype=object)

In [8]:
IAS_RISK = []

for i in df1['RISK_TIER']:
    if i == 'moderate':
        IAS_RISK.append('Medium')
    else:
        IAS_RISK.append('High')

In [9]:
df1['IAS_RISK'] = IAS_RISK

In [10]:
df2 = df1[['VIDEO_ID', 'CHANNEL_ID', 'VIDEO_CATEGORY', 'IAS_RISK']]

In [15]:
df2.head()

,VIDEO_ID,CHANNEL_ID,VIDEO_CATEGORY,IAS_RISK
0,xr9GGusfrW4,UCpB959t8iPrxQWj7G6n0ctQ,Entertainment,Medium
1,ptvqJBL4ljA,UCl6UMZgAD7EDCNTB8Y-ICiQ,Education,Medium
2,_-3v1MonI54,UCNIFiHaLZkYASaWDdkC1njg,Entertainment,Medium
3,kPg3M4C9N9w,UCmBA_wu8xGg1OfOkfW13Q0Q,Music,Medium
4,ws00k_lIQ9U,UCmBA_wu8xGg1OfOkfW13Q0Q,Music,Medium


In [11]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import pd_writer

In [12]:
table_name = 'IAS_Q122'
schema = 'ANALYST'

In [13]:
def connection(schema):
    conn = snowflake.connector.connect(
        host="localhost",
        port="10015",
        user="sdm",
        password="sdm",
        account="dummy",
        protocol="http",
        warehouse="ENGINEER_WH",
        database="PROD",
        schema=schema,
    )
    return conn

In [14]:
def write_snowflake(schema,df,table_name):
    conn = connection(schema)
    success, nchunks, nrows, _ = write_pandas(conn, df, table_name, chunk_size = 1000)
    
    return success, nchunks, nrows, _

In [16]:
success, nchunks, nrows, _ = write_snowflake(schema, df2, table_name)